Must be run in Python 3

In [12]:
from keras.models import Model
from keras.layers import Embedding, Dense, Input, Reshape, Flatten, dot, Add
import numpy as np
import keras.backend as K
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import itertools
import tensorflow as tf
import sys

In [13]:
wiki_model = Word2Vec.load("english-wikipedia-articles-20170820-models/enwiki_2017_08_20_fasttext.model")

In [14]:
vocab_size = 22000
vector_dim = 100

In [15]:
def convert_np_to_bin_model(np_vectors,model_name):
    with open(model_name+'.txt', 'w') as we:
        we.write('{} {}\n'.format(vocab_size,vector_dim))
        for i in range(vocab_size):
            w = wiki_model.wv.index2word[i]
            vec = np_vectors[i]
            we.write('{} '.format(w))
            for v in vec:
                we.write(str(v) + ' ')
            we.write('\n')
    model = KeyedVectors.load_word2vec_format(model_name+'.txt', binary=False)
    model.save_word2vec_format(model_name+'.bin', binary=True)
    print('created model '+model_name+'.bin')

In [5]:
wvs_embed_trained_normalized_embeddings = np.load('wvs_embed_trained_normalized_embeddings.npy')

In [7]:
convert_np_to_bin_model(wvs_embed_trained_normalized_embeddings,'wvs_embed_trained_normalized_embeddings')

created model wvs_embed_trained_normalized_embeddings_2.bin


In [27]:
! ../word-embeddings-benchmarks/scripts/evaluate_on_all_orig.py -f original_embed_small.bin

original_embed_small.bin
12:51:34 INFO:loading projection weights from /Users/haileyjames/Documents/wasserstein-debiasing/original_embed_small.bin
12:51:34 INFO:Loading #22000 words with 100 dim
12:51:34 INFO:Transformed 22000 into 22000 words
12:51:34 INFO:Calculating similarity benchmarks
12:51:34 WARNING:Missing 531 words. Will replace them with mean vector
12:51:34 INFO:Spearman correlation of scores on MEN 0.5983153481584027
12:51:34 WARNING:Missing 48 words. Will replace them with mean vector
12:51:34 INFO:Spearman correlation of scores on WS353 0.46266864889490805
12:51:34 WARNING:Missing 27 words. Will replace them with mean vector
12:51:34 INFO:Spearman correlation of scores on WS353R 0.4104594294511796
12:51:34 WARNING:Missing 27 words. Will replace them with mean vector
12:51:34 INFO:Spearman correlation of scores on WS353S 0.5393883796332126
12:51:34 WARNING:Missing 93 words. Will replace them with mean vector
12:51:34 INFO:Spearman correlation of scores on SimLex999 0.2548

In [16]:
! ../word-embeddings-benchmarks/scripts/evaluate_on_all_orig.py -f wvs_embed_trained_normalized_embeddings.bin

wvs_embed_trained_normalized_embeddings.bin
09:07:23 INFO:loading projection weights from /Users/haileyjames/Documents/wasserstein-debiasing/wvs_embed_trained_normalized_embeddings.bin
09:07:23 INFO:Loading #22000 words with 100 dim
09:07:23 INFO:Transformed 22000 into 22000 words
09:07:23 INFO:Calculating similarity benchmarks
09:07:23 WARNING:Missing 531 words. Will replace them with mean vector
09:07:23 INFO:Spearman correlation of scores on MEN 0.5775930360006853
09:07:23 WARNING:Missing 48 words. Will replace them with mean vector
09:07:23 INFO:Spearman correlation of scores on WS353 0.3902896158576897
09:07:23 WARNING:Missing 27 words. Will replace them with mean vector
09:07:23 INFO:Spearman correlation of scores on WS353R 0.3472629665223466
09:07:23 WARNING:Missing 27 words. Will replace them with mean vector
09:07:23 INFO:Spearman correlation of scores on WS353S 0.4755790923001263
09:07:23 WARNING:Missing 93 words. Will replace them with mean vector
09:07:23 INFO:Spearman corr

Change `compare_embeddings_gender.config` to have the name of the embeddings

In [17]:
!python ../compare-embedding-bias/weat.py ../compare-embedding-bias/configs/compare_embeddings_gender.json

Original Embedding
+-------------------------------------------------------+-------+-------+
|                         Test                          | Mean  | Error |
+-------------------------------------------------------+-------+-------+
| Flowers vs Insects (25) / Pleasant vs Unpleasant (25) | 1.395 | 0.036 |
+-------------------------------------------------------+-------+-------+
|   Instruments vs Weapons /  Pleasant vs Unpleasant    | 1.505 | 0.015 |
+-------------------------------------------------------+-------+-------+
|         Male8 vs Female8 /  Career vs Family          | 1.052 | 0.042 |
+-------------------------------------------------------+-------+-------+
|            Math vs Art / Male8 vs Female8             | 1.061 | 0.061 |
+-------------------------------------------------------+-------+-------+
|           Science vs Art / Male8 vs Female8           | 1.204 | 0.064 |
+-------------------------------------------------------+-------+-------+
Debiased Embedding
